In [52]:
'''
Created by: Kristen McGarry, EDO Data Analyst Intern
Date: July 2017
Last Updated: July 27, 2017
'''

'\nCreated by: Kristen McGarry, EDO Data Analyst Intern\nDate: July 2017\nLast Updated: July 27, 2017\n'

In [53]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import codecs
import string
import re

In [54]:
# REMOVE HTML TAGS

def cleanhtml(raw_html,outFile):
    table_cleaner = re.compile('(?i)<table.*?</table>')
    regexs = [table_cleaner, '<.*?>', '\n', '\t', '&nbsp', '&#']
    for regex in regexs:
        raw_html = re.sub(regex, ' ', raw_html)

    raw_html = raw_html.replace(";"," ")
        
    out_file = open(outFile,"w")
    out_file.write(raw_html)
    out_file.close()
    
    return "done"


In [55]:
# REMOVE NON-ASCII CHARACTERS

def onlyAscii(inFile,outFile):
    ip=open(inFile,'r')
    op=open(outFile,'w')
    for line in ip:
        line=line.strip().decode("ascii","ignore").encode("ascii")
        whitespace = "\r\n\t"
        line = line.strip(whitespace)
        if line=="":continue
        op.write(line)
    ip.close()
    op.close()

In [56]:
# READ IN TEXT WITH REMOVED TAGS AND NON-ASCII CHARACTERS

def getText(inFile):
    i_file = open(inFile,'r')
    text = i_file.read()
    return text

In [57]:
# REMOVE STOP WORDS

def removeStopWords(text):
    stop = (stopwords.words('english')) + (list(string.punctuation))

    clean_text = []
    tokens = word_tokenize(text.lower())

    for word in tokens:
        if word not in stop:
            clean_text.append(word)

    return clean_text

In [58]:
# WORD FREQUENCY

def wordFrequency(clean_text):
    freq_dict = {}

    for word in clean_text:
        if word in freq_dict:
            freq_dict[word] = freq_dict[word] + 1
        else:
            freq_dict[word] =1
    return freq_dict

In [59]:
# SORT WORD FREQUENCY IN DESCENDING ORDER

def sortWordFreq(freq_dict):
    
    top_value = 0
    top_word = ""
    mybool = True
    
    while mybool == True:
        for word in freq_dict.keys():
            value = freq_dict[word]
            if value > top_value:
                top_value = value
                top_word = word
            else:
                continue

        print (top_word + ": " + str(top_value))
        
        del freq_dict[top_word]
        top_value = 0
        top_word = ""
        
        if (len(freq_dict.keys()) == 0):
            mybool = False
        else:
            continue
    
            

In [60]:
# SENTENCE TOKENIZER

def tokenizer(text):
    sent_tokens = nltk.sent_tokenize(text)
    return sent_tokens

In [61]:
# PART OF SPEECH (POS) TAGGER

def posTagger(sent_tokens):
    pos_text = []

    for i in range(0,(len(sent_tokens))):
        word_tokens = word_tokenize(sent_tokens[i])
        pos_temp = nltk.pos_tag(word_tokens)
        pos_text.append(pos_temp)
    return pos_text

In [62]:
# EXTRACT CERTAIN POS TAGS

def cleanPosTagger(pos_tags):
    #key_tags = ["CD","NNP","VB","VBD","VBG","VBN","VBP","VBZ"]
    key_tags = ["NNP"]
    key_text = ""
    key_text_tags = []
    
    for sent in pos_tags:
        for word_tag in sent:
            if word_tag[1] in key_tags:
                if word_tag[0] != "|":
                    key_text_tags += word_tag
                    key_text = key_text + word_tag[0] + " "
    return key_text

In [63]:
# REMOVE REPEATING WORDS (i.e. board board directors)

def removeRepeats(key_text):
    text = ""
    words = key_text.split(" ")
    for i in range(0,len(words)-1):
        if len(words[i]) > 2:
            if words[i] != words[i+1]:
                text =  text + " " + words[i]
        else:
            continue
    return text

In [73]:
# NEO4J CYPHER COMMANDS

def neo4j(key_tag_text, list_directors,cypher_file):
    out_file = open(cypher_file,"w")
    
    create_words = ('WITH split(tolower("' + key_tag_text + '")," ") AS text UNWIND range(0,size(text)-2) AS i MERGE(w1:Word {name :text[i]}) ON CREATE SET w1.count = 1 ON MATCH SET w1.count = w1.count + 1 MERGE(w2:Word {name :text[i+1]}) ON CREATE SET w2.count = 1 ON MATCH SET w2.count = w2.count + 1 MERGE(w1)-[r:NEXT]->(w2) ON CREATE SET r.count = 1 ON MATCH SET r.count = r.count + 1;')
    out_file.write(create_words)
    out_file.write('\n')
    
    # HARD CODED TO APPLE DOC
    create_doc = 'CREATE (Apple14A:Document{title:"Apple Inc. Notice of 2017 Annual Meeing of Shareholders and Proxy Statement", type:"14A", date:"2017-02-28", source:"SEC EDGAR"});'
    out_file.write(create_doc)
    out_file.write('\n')
    
    for director in list_directors:
        names = director.split(" ")
        for name in names:
            name = name.lower()
            if len(name) > 2: 
                add_person_label = 'MATCH (' + name + ':Word) WHERE ' + name + '.name = "' + name + '" SET ' + name + ':Person;'
                out_file.write(add_person_label)
                out_file.write('\n')
                
    out_file.close()

In [74]:
# COMMANDS

In [75]:
in_file = open("C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph.htm","r")
raw_html = in_file.read()
cleanhtml(raw_html,"C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph.txt")


'done'

In [76]:
full_text = getText("C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph.txt")
words = removeStopWords(full_text)
sortWordFreq(wordFrequency(words))

board: 44
executive: 28
company: 27
apple: 25
chief: 23
officer: 23
served: 23
experience: 20
directors: 20
committee: 19
inc.: 19
proxy: 18
since: 17
public: 16
146: 16
chairman: 15
directorships: 14
president: 14
financial: 14
nominating: 14
years: 13
mr.: 12
director: 12
access: 11
management: 10
leadership: 10
international: 9
shareholders: 9
among: 9
vice: 9
business: 8
brings: 8
including: 8
service: 8
candidates: 8
qualifications: 8
old: 8
trustees: 7
corporate: 7
cook: 6
large: 6
selected: 6
ms.: 6
149: 6
2017: 6
statement: 6
memberships: 6
current: 6
blackrock: 6
nominees: 6
amp: 6
extensive: 6
shareholder: 6
operating: 6
2000: 6
global: 5
shares: 5
dr.: 5
along: 5
september: 5
2012: 5
contents: 5
senior: 5
sugar: 5
wagner: 5
avon: 5
april: 5
bell: 5
table: 5
member: 5
jung: 5
previously: 5
gore: 5
expertise: 5
positions: 4
1998: 4
products: 4
governance: 4
council: 4
operations: 4
university: 4
members: 4
nominate: 4
also: 4
worldwide: 4
levinson: 4
advisory: 4
disney: 4
form

In [77]:
onlyAscii("C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph.txt","C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph-ascii.txt")

In [78]:
clean_text = open("C:/dev/EDO-Intern-Project/Kristen/nlp/data/appl-secfiling-director-paragraph-ascii.txt","r").readline()
sent_tokens = tokenizer(clean_text)
pos_tags = posTagger(sent_tokens)

In [79]:
key_text = cleanPosTagger(pos_tags)


In [80]:
nnp_text = removeRepeats(key_text)

In [82]:
listDirectors = ["Tim Cook", "Arthur Levinson","James Bell", "Albert Gore", "Robert Iger", "Andrea Jung","Ronald Sugar","Susan Wagner"]
neo4j(nnp_text, listDirectors, "C:/dev/EDO-Intern-Project/Kristen/nlp/data/cypher-commands-v2.txt")